In [1]:
import xsimlab as xs
import xarray as xr
from episimlab.models import ExampleSIR, EpiModel
from episimlab.models.example_sir import SetupPhi
from episimlab.models.partition_v1 import *
from episimlab.compt_model import ComptModel
from episimlab.foi import BaseFOI
from episimlab.utils import get_var_dims, group_dict_by_var, visualize_compt_graph, coerce_to_da, fix_coord_dtypes, IntPerDay
from episimlab.setup.sto import SetupStochasticFromToggle
from episimlab.setup.seed import SeedGenerator
import networkx as nx
import pandas as pd
import numpy as np
import math
import datetime
import geopandas as gpd

/Users/kpierce/epimodels/epi-venv/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# ZCTA-level high risk proportion for the state of Texas

## Goal: parameterize Austin granular model with ZCTA level data on risk percentage.

### Risk data

We have CDC 500 Cities places data at the Census Block Group level that could be used to make risk proportion calculations. We also have pre-calculated risk percentages courtesy of Remy Pasco, which is what we'll use moving forward.

The raw data that we won't use:

In [2]:
places = pd.read_csv(
    '../CDC_PLACES_DATA/500_Cities__Local_Data_for_Better_Health__2019_release.csv',
    dtype={'TractFIPS': str, 'CityFIPS': str}
)

In [3]:
places['GeographicLevel'].value_counts()

Census Tract    782047
City             28000
US                  56
Name: GeographicLevel, dtype: int64

Remy's risk data, which we will use:

In [4]:
zcta_age_risk = pd.read_csv(
    '../Remy_ZCTA_high_risk/COVID High risk population per age group per zip code.csv',
    dtype={'ZCTA5': str}
)

In [5]:
zcta_age_risk.head()

,ZCTA5,0_0.5,0.5_4,5_9,10_14,15_19,20_24,25_29,30_34,35_39,40_44,45_49,50_54,55_59,60_64,65_69,70_74,75+
0,1086,0.0,0.063143,0.092746,0.104584,0.117104,0.135435,0.147714,0.147180,0.131456,0.116898,0.153668,0.153505,0.153505,0.153505,0.198322,0.198322,0.198322
1,4469,0.0,0.090581,0.133047,0.149925,0.163973,0.184093,0.194950,0.195366,0.179244,0.168342,0.227767,0.227767,0.227767,0.227767,0.294302,0.294302,0.294302
2,6269,0.0,0.082688,0.121454,0.136861,0.148272,0.157956,0.183474,0.196116,0.175064,0.158072,0.217177,0.216874,0.216874,0.216874,0.283603,0.283603,0.283603
3,8240,0.0,0.058633,0.086122,0.097047,0.108229,0.127235,0.143871,0.145469,0.125942,0.110078,0.143980,0.143747,0.143747,0.143747,0.186270,0.186270,0.186270
4,11549,0.0,0.055626,0.081705,0.092094,0.100741,0.112221,0.130361,0.155854,0.135504,0.108977,0.142456,0.141966,0.141889,0.141889,0.186404,0.186404,0.186404


Note that there is 0% risk percentage in the 0-0.5y age bin.

In [6]:
max(zcta_age_risk['0_0.5'])

0.0

In [7]:
zcta_age_risk_long = pd.melt(zcta_age_risk, id_vars='ZCTA5', var_name='age_group', value_name='prop_high_risk')

In [8]:
zcta_age_risk_long.head()

,ZCTA5,age_group,prop_high_risk
0,1086,0_0.5,0.0
1,4469,0_0.5,0.0
2,6269,0_0.5,0.0
3,8240,0_0.5,0.0
4,11549,0_0.5,0.0


In [9]:
def split_age_group(long_df, split_grp, newgrp1, newgrp2):
    
    # only works if the age column is 'age_group'
    assert 'age_group' in long_df.columns

    # separate age groups
    split_grp_df = long_df[(long_df['age_group'] == split_grp)]
    other_groups = long_df[(long_df['age_group'] != split_grp) ]
    
    # copy dataframes
    group_1 = deepcopy(split_grp_df)
    group_2 = deepcopy(split_grp_df)

    # reassign age group 1
    group_1['age_group'] = [newgrp1 if i == split_grp else i for i in group_1['age_group']]
    
    # reassign age group 2
    group_2['age_group'] = [newgrp2 if i == split_grp else i for i in group_2['age_group']]
    
    # join everything
    regrouped = pd.concat([group_1, group_2, other_groups])
    
    return regrouped

In [10]:
from copy import deepcopy

In [11]:
zcta_risk_split = split_age_group(zcta_age_risk_long, split_grp='10_14', newgrp1='10', newgrp2='11_14')
zcta_risk_split = split_age_group(zcta_risk_split, split_grp='11_14', newgrp1='11_13', newgrp2='14')
zcta_risk_split = split_age_group(zcta_risk_split, split_grp='15_19', newgrp1='15_17', newgrp2='18_19')

In [12]:
zcta_risk_split['age_group'].value_counts()

15_17    32409
18_19    32409
70_74    32409
65_69    32409
60_64    32409
55_59    32409
50_54    32409
45_49    32409
40_44    32409
35_39    32409
30_34    32409
25_29    32409
20_24    32409
5_9      32409
0.5_4    32409
0_0.5    32409
10       32409
14       32409
11_13    32409
75+      32409
Name: age_group, dtype: int64

In [13]:
revised_groups = {
    '0_0.5': '00-04',
    '0.5_4': '00-04',
    '5_9': '05-10',
    '10': '05-10',
    '11_13': '11-13',
    '14': '14-17',
    '15_17': '14-17',
    '18_19': '18-49',
    '20_24': '18-49',
    '25_29': '18-49',
    '30_34': '18-49',
    '35_39': '18-49',
    '40_44': '18-49',
    '45_49': '18-49',
    '50_54': '50-64',
    '55_59': '50-64',
    '60_64': '50-64',
    '65_69': '65+',
    '70_74': '65+',
    '75+': '65+'
}

In [14]:
zcta_risk_split['revised_age_group'] = [revised_groups[i] for i in zcta_risk_split['age_group']]

In [15]:
zcta_risk_split.head()

,ZCTA5,age_group,prop_high_risk,revised_age_group
129636,1086,15_17,0.117104,14-17
129637,4469,15_17,0.163973,14-17
129638,6269,15_17,0.148272,14-17
129639,8240,15_17,0.108229,14-17
129640,11549,15_17,0.100741,14-17


In [16]:
us_pop_long_group_totals = pd.read_csv('/Users/kpierce/epimodels/notebooks/AustinGranularModel/DataIntegration/us_pop_by_pct_revised_age_group.csv')


In [17]:
us_pop_long_group_totals.head()

,Unnamed: 0,age,census_age_group,revised_age_group,year,population,census_age_group_total,revised_age_group_total,age_pct_of_census_group,age_pct_of_revised_group
0,0,0,00-04,00-04,2010,3951430,20188815,20188815,0.195724,0.195724
1,1,1,00-04,00-04,2010,3957730,20188815,20188815,0.196036,0.196036
2,2,2,00-04,00-04,2010,4090621,20188815,20188815,0.202618,0.202618
3,3,3,00-04,00-04,2010,4111688,20188815,20188815,0.203662,0.203662
4,4,4,00-04,00-04,2010,4077346,20188815,20188815,0.201961,0.201961


In [18]:
us_pop_long_group_totals = us_pop_long_group_totals[us_pop_long_group_totals['year'].isin(set([2018, 2019]))]

In [19]:
zcta_risk = pd.merge(
    zcta_risk_split.drop(columns=['age_group']),
    us_pop_long_group_totals,
    on='revised_age_group',
    how='left'
)

In [20]:
zcta_risk.head()

,ZCTA5,prop_high_risk,revised_age_group,Unnamed: 0,age,census_age_group,year,population,census_age_group_total,revised_age_group_total,age_pct_of_census_group,age_pct_of_revised_group
0,1086,0.117104,14-17,822,14,10-14,2018,4164608,20868629,16663877,0.199563,0.249918
1,1086,0.117104,14-17,823,15,15-17,2018,4137711,12499269,16663877,0.331036,0.248304
2,1086,0.117104,14-17,824,16,15-17,2018,4125819,12499269,16663877,0.330085,0.247591
3,1086,0.117104,14-17,825,17,15-17,2018,4235739,12499269,16663877,0.338879,0.254187
4,1086,0.117104,14-17,923,14,10-14,2019,4164459,20798268,16632763,0.200231,0.250377


In [27]:
zcta_risk['adjusted_risk_age_numerator'] = zcta_risk['age_pct_of_revised_group'] * zcta_risk['prop_high_risk']
zcta_risk_weighted = zcta_risk.groupby(
    ['revised_age_group', 'ZCTA5', 'year']
).sum(
    ['adjusted_risk_age_numerator', 'age_pct_of_revised_group']
).reset_index()

In [28]:
zcta_risk_weighted['weighted_pct_high_risk'] = zcta_risk_weighted['adjusted_risk_age_numerator'] / zcta_risk_weighted['age_pct_of_revised_group']


In [29]:
zcta_risk_weighted.head()

,revised_age_group,ZCTA5,year,prop_high_risk,Unnamed: 0,age,population,census_age_group_total,revised_age_group_total,age_pct_of_census_group,age_pct_of_revised_group,adjusted_risk_age_numerator,weighted_pct_high_risk
0,00-04,10001,2018,0.321354,8100,20,39525924,197629620,197629620,2.0,2.0,0.064271,0.032135
1,00-04,10001,2019,0.321354,9110,20,39153366,195766830,195766830,2.0,2.0,0.064271,0.032135
2,00-04,10002,2018,0.362698,8100,20,39525924,197629620,197629620,2.0,2.0,0.072540,0.036270
3,00-04,10002,2019,0.362698,9110,20,39153366,195766830,195766830,2.0,2.0,0.072540,0.036270
4,00-04,10003,2018,0.285648,8100,20,39525924,197629620,197629620,2.0,2.0,0.057130,0.028565


In [30]:
max(zcta_age_risk_long['prop_high_risk'])

0.889079677153696

In [31]:
max(zcta_risk_weighted['weighted_pct_high_risk'])

0.8890796771536961

In [32]:
zcta_risk_weighted['revised_age_group'].unique()

array(['00-04', '05-10', '11-13', '14-17', '18-49', '50-64', '65+'],
      dtype=object)

In [35]:
zcta_risk_weighted[['revised_age_group', 'ZCTA5', 'weighted_pct_high_risk', 'year']].to_csv(
    '/Users/kpierce/epimodels/notebooks/AustinGranularModel/DataIntegration/zcta_age_groups_risk_prop_2018_2019.csv'
)

### Travel data

In [28]:
zcta_pop = pd.read_csv(
    '/Users/kpierce/COVID19/SchoolCatchmentDemo/2019_TX_ZCTA_age_populations.csv',
    dtype={'GEOID': str}
)

In [29]:
zcta_pop.head()

,Unnamed: 0,GEOID,age,estimate
0,1,75001,10 to 14 years,401
1,2,75001,15 to 17 years,277
2,3,75001,18 and 19 years,147
3,4,75001,20 years,180
4,5,75001,21 years,163


In [30]:
zcta_pop['age'].unique()

array(['10 to 14 years', '15 to 17 years', '18 and 19 years', '20 years',
       '21 years', '22 to 24 years', '25 to 29 years', '30 to 34 years',
       '35 to 39 years', '40 to 44 years', '45 to 49 years',
       '5 to 9 years', '50 to 54 years', '55 to 59 years',
       '60 and 61 years', '62 to 64 years', '65 and 66 years',
       '67 to 69 years', '70 to 74 years', '75 to 79 years',
       '80 to 84 years', '85 years and over', 'Under 5 years'],
      dtype=object)

### Merge risk and population data

Risk data and population data have different binning strategies; we need to map one binning strategy on to the other and merge the data sets.

In [31]:
pop_risk_map = {
    '10 to 14 years': '10_14',
    '15 to 17 years': '15_19',
    '18 and 19 years': '15_19',
    '20 years': '20_24',
    '21 years': '20_24', 
    '22 to 24 years': '20_24', 
    '25 to 29 years': '25_29', 
    '30 to 34 years': '30_34',
    '35 to 39 years': '35_39', 
    '40 to 44 years': '40_44', 
    '45 to 49 years': '45_49',
    '5 to 9 years': '5_9', 
    '50 to 54 years': '50_54', 
    '55 to 59 years': '55_59',
    '60 and 61 years': '60_64', 
    '62 to 64 years': '60_64', 
    '65 and 66 years': '65_69',
    '67 to 69 years': '65_69', 
    '70 to 74 years': '70_74', 
    '75 to 79 years': '75+',
    '80 to 84 years': '75+', 
    '85 years and over': '75+', 
    'Under 5 years': '0.5_4' # risk is zero for 0-0.5 years, so we can ignore that category
    
}

In [32]:
zcta_pop['age_bin'] = [pop_risk_map[i] for i in zcta_pop['age']]

In [33]:
zcta_pop.head()

,Unnamed: 0,GEOID,age,estimate,age_bin
0,1,75001,10 to 14 years,401,10_14
1,2,75001,15 to 17 years,277,15_19
2,3,75001,18 and 19 years,147,15_19
3,4,75001,20 years,180,20_24
4,5,75001,21 years,163,20_24


In [34]:
zcta_pop_total = zcta_pop.groupby(['GEOID', 'age_bin']).sum('estimate').reset_index().drop('Unnamed: 0', axis=1)

In [35]:
zcta_pop_total.head()

,GEOID,age_bin,estimate
0,75001,0.5_4,794
1,75001,10_14,401
2,75001,15_19,424
3,75001,20_24,1276
4,75001,25_29,2859


There should be 1,939 ZCTAs in Texas, so we're missing a couple:

In [50]:
len(zcta_pop_total['GEOID'].unique())

1935

In [36]:
zcta_age_risk_texas = zcta_age_risk[zcta_age_risk['ZCTA5'].isin(set(zcta_pop_total['GEOID'].unique()))]

In [52]:
len(zcta_age_risk_texas['ZCTA5'].unique())

1896

In [37]:
zcta_age_risk.shape

(32409, 18)

In [38]:
zcta_age_risk_texas.shape

(1896, 18)

In [40]:
zcta_age_risk_texas_long = pd.melt(zcta_age_risk_texas, id_vars='ZCTA5')

In [41]:
zcta_age_risk_texas_long.head()

,ZCTA5,variable,value
0,75962,0_0.5,0.0
1,76402,0_0.5,0.0
2,76798,0_0.5,0.0
3,76908,0_0.5,0.0
4,78712,0_0.5,0.0


In [43]:
zcta_age_pct_risk = pd.merge(
    zcta_age_risk_texas_long, 
    zcta_pop_total, left_on=['ZCTA5', 'variable'], 
    right_on=['GEOID', 'age_bin']
)

In [55]:
zcta_age_pct_risk = zcta_age_pct_risk.rename(
    columns={
        'value': 'pct_high_risk',
        'estimate': 'total_population'
    }
)
zcta_age_pct_risk = zcta_age_pct_risk.drop(['ZCTA5', 'variable'], axis=1)

In [56]:
zcta_age_pct_risk.head()

,pct_high_risk,GEOID,age_bin,total_population
0,0.108621,75962,0.5_4,0
1,0.090956,76402,0.5_4,0
2,0.103360,76798,0.5_4,0
3,0.079681,76908,0.5_4,0
4,0.086070,78712,0.5_4,0


In [59]:
zcta_age_pct_risk['high_risk_population'] = zcta_age_pct_risk['pct_high_risk'] * zcta_age_pct_risk['total_population']

We next do a final widening of age bins. Past models have used five age bins; the current binning strategy for these data does not allow us to exactly match those bin widths, but we can get close.

In [70]:
# '0-4', '5-19', '20-49', '50-64', '65+' -- slightly different from usual age groups
wide_age_bin_mapping = {
    '10_14': '5-19',
    '15_19': '5-19',
    '20_24': '20-49', 
    '25_29': '20-49', 
    '30_34': '20-49',
    '35_39': '20-49', 
    '40_44': '20-49', 
    '45_49': '20-49',
    '5_9': '5-19', 
    '50_54': '50-64', 
    '55_59': '50-64',
    '60_64': '50-64', 
    '65_69': '65+',
    '70_74': '65+', 
    '75+': '65+',
    '0.5_4': '0-4' # risk is zero for 0-0.5 years, so we can ignore that category   
}

In [71]:
zcta_age_pct_risk['wide_age_bin'] = [wide_age_bin_mapping[i] for i in zcta_age_pct_risk['age_bin']]

In [72]:
zcta_wide_age_pct_risk = zcta_age_pct_risk.groupby(['GEOID', 'wide_age_bin']).sum(['total_population', 'high_risk_population']).reset_index().drop('pct_high_risk', axis=1)


In [73]:
zcta_wide_age_pct_risk.head()

,GEOID,wide_age_bin,total_population,high_risk_population
0,75001,0-4,794,87.140716
1,75001,20-49,9273,2546.867010
2,75001,5-19,1551,282.514614
3,75001,50-64,2259,848.093682
4,75001,65+,1115,574.459288


Simple arithmetic to get the final dataset, then save:

In [74]:
zcta_wide_age_pct_risk['pct_high_risk'] = zcta_wide_age_pct_risk['high_risk_population']/zcta_wide_age_pct_risk['total_population']

In [77]:
zcta_wide_age_pct_risk['low_risk_population'] = zcta_wide_age_pct_risk['total_population'] - zcta_wide_age_pct_risk['high_risk_population']

In [78]:
zcta_wide_age_pct_risk.head()

,GEOID,wide_age_bin,total_population,high_risk_population,pct_high_risk,low_risk_population
0,75001,0-4,794,87.140716,0.109749,706.859284
1,75001,20-49,9273,2546.867010,0.274654,6726.132990
2,75001,5-19,1551,282.514614,0.182150,1268.485386
3,75001,50-64,2259,848.093682,0.375429,1410.906318
4,75001,65+,1115,574.459288,0.515210,540.540712


In [79]:
zcta_wide_age_pct_risk['wide_age_bin'].unique()

array(['0-4', '20-49', '5-19', '50-64', '65+'], dtype=object)

In [80]:
zcta_wide_age_pct_risk.to_csv('/Users/kpierce/epimodels/sandbox-scripts/tx_zcta_high_risk.csv')